In [2]:
'''
MIT License
Copyright (c) 2019 Fanjin Zeng
This work is licensed under the terms of the MIT license, see <https://opensource.org/licenses/MIT>.  
https://gist.github.com/Fnjn/58e5eaa27a3dc004c3526ea82a92de80
RRT = rapidly exploring random tree
'''

import numpy as np
from random import random
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
from collections import deque
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import ipympl
import scipy.io
%matplotlib widget

In [3]:
# 3D version in progress
def dijkstra(G):
    '''
    Dijkstra algorithm for finding shortest path from start position to end.
    '''
    srcIdx = G.vex2idx[G.startpos]
    dstIdx = G.vex2idx[G.endpos]

    # build dijkstra
    nodes = list(G.neighbors.keys())
    dist = {node: float('inf') for node in nodes}
    prev = {node: None for node in nodes}
    dist[srcIdx] = 0

    while nodes:
        curNode = min(nodes, key=lambda node: dist[node])
        nodes.remove(curNode)
        if dist[curNode] == float('inf'):
            break

        for neighbor, cost in G.neighbors[curNode]:
            newCost = dist[curNode] + cost
            if newCost < dist[neighbor]:
                dist[neighbor] = newCost
                prev[neighbor] = curNode

    # retrieve path
    path = deque()
    curNode = dstIdx
    while prev[curNode] is not None:
        path.appendleft(G.vertices[curNode])
        curNode = prev[curNode]
    path.appendleft(G.vertices[curNode])
    return list(path)

class Line():
    ''' Define line '''
    def __init__(self, p0, p1):
        self.p = np.array(p0)
        self.dirn = np.array(p1) - np.array(p0)
        self.dist = np.linalg.norm(self.dirn)
        self.dirn /= self.dist # normalize

    def path(self, t):
        return self.p + t * self.dirn


def Intersection(line, center, radius):
    ''' Check line-sphere (circle) intersection '''
    a = np.dot(line.dirn, line.dirn)
    b = 2 * np.dot(line.dirn, line.p - center)
    c = np.dot(line.p - center, line.p - center) - radius * radius

    discriminant = b * b - 4 * a * c
    if discriminant < 0:
        return False

    t1 = (-b + np.sqrt(discriminant)) / (2 * a);
    t2 = (-b - np.sqrt(discriminant)) / (2 * a);

    if (t1 < 0 and t2 < 0) or (t1 > line.dist and t2 > line.dist):
        return False

    return True


def distance(obj1, obj2):
    return np.linalg.norm(np.array(obj1) - np.array(obj2))


#vertex
def isInObstacle(vex, obstacles, radius):
    for obs in obstacles:
        if distance(obs, vex) < radius:
            return True
    return False



#
def isThruObstacle(line, obstacles, radius):
    for obs in obstacles:
        if Intersection(line, obs, radius):
            return True
    return False


def nearest(G, vex, obstacles, radius):
    Nvex = None
    Nidx = None
    minDist = float("inf")

    for idx, v in enumerate(G.vertices):
        line = Line(v, vex)
        if isThruObstacle(line, obstacles, radius):
            continue

        dist = distance(v, vex)
        if dist < minDist:
            minDist = dist
            Nidx = idx
            Nvex = v

    return Nvex, Nidx


def newVertex(randvex, nearvex, stepSize):
    dirn = np.array(randvex) - np.array(nearvex)
    length = np.linalg.norm(dirn)
    dirn = (dirn / length) * min (stepSize, length)

    newvex = (nearvex[0]+dirn[0], nearvex[1]+dirn[1], nearvex[2] + dirn[2])
    return newvex


def window(startpos, endpos):
    ''' Define seach window - 2 times of start to end rectangle'''
    width = endpos[0] - startpos[0]
    height = endpos[1] - startpos[1]
    winx = startpos[0] - (width / 2.)
    winy = startpos[1] - (height / 2.)
    return winx, winy, width, height


def isInWindow(pos, winx, winy, width, height):
    ''' Restrict new vertex insides search window'''
    if winx < pos[0] < winx+width and \
        winy < pos[1] < winy+height:
        return True
    else:
        return False


class Graph:
    ''' Define graph '''
    def __init__(self, startpos, endpos):
        self.startpos = startpos
        self.endpos = endpos

        self.vertices = [startpos]
        self.edges = []
        self.success = False

        self.vex2idx = {startpos:0}
        self.neighbors = {0:[]}
        self.distances = {0:0.}

        self.sx = endpos[0] - startpos[0]
        self.sy = endpos[1] - startpos[1]
        self.sz = endpos[2] - startpos[2]

    def add_vex(self, pos):
        try:
            idx = self.vex2idx[pos]
        except:
            idx = len(self.vertices)
            self.vertices.append(pos)
            self.vex2idx[pos] = idx
            self.neighbors[idx] = []
        return idx

    def add_edge(self, idx1, idx2, cost):
        self.edges.append((idx1, idx2))
        self.neighbors[idx1].append((idx2, cost))
        self.neighbors[idx2].append((idx1, cost))


    def randomPosition(self):
        rx = random()
        ry = random()
        rz = random()

        posx = self.startpos[0] - (self.sx / 2.) + rx * self.sx * 2
        posy = self.startpos[1] - (self.sy / 2.) + ry * self.sy * 2
        posz = self.startpos[2] - (self.sz / 2.) + rz * self.sz * 2
        return posx, posy, posz


def RRT(startpos, endpos, obstacles, n_iter, radius, stepSize):
    ''' RRT algorithm '''
    G = Graph(startpos, endpos)

    for _ in range(n_iter):
        randvex = G.randomPosition() #random graph
        if isInObstacle(randvex, obstacles, radius):
            continue

        nearvex, nearidx = nearest(G, randvex, obstacles, radius)
        if nearvex is None:
            continue

        newvex = newVertex(randvex, nearvex, stepSize)

        newidx = G.add_vex(newvex)
        dist = distance(newvex, nearvex)
        G.add_edge(newidx, nearidx, dist)

        dist = distance(newvex, G.endpos)
        if dist < 2 * radius:
            endidx = G.add_vex(G.endpos)
            G.add_edge(newidx, endidx, dist)
            G.success = True
            #print('success')
            # break
    return G


def RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize):
    ''' RRT star algorithm '''
    G = Graph(startpos, endpos)

    for _ in range(n_iter):
        randvex = G.randomPosition()
        if isInObstacle(randvex, obstacles, radius):
            continue

        nearvex, nearidx = nearest(G, randvex, obstacles, radius)
        if nearvex is None:
            continue

        newvex = newVertex(randvex, nearvex, stepSize)

        newidx = G.add_vex(newvex)
        dist = distance(newvex, nearvex)
        G.add_edge(newidx, nearidx, dist)
        G.distances[newidx] = G.distances[nearidx] + dist

        # update nearby vertices distance (if shorter)
        for vex in G.vertices:
            if vex == newvex:
                continue

            dist = distance(vex, newvex)
            if dist > radius:
                continue

            line = Line(vex, newvex)
            if isThruObstacle(line, obstacles, radius): #check if a line
                continue

            idx = G.vex2idx[vex]
            if G.distances[newidx] + dist < G.distances[idx]:
                G.add_edge(idx, newidx, dist)
                G.distances[idx] = G.distances[newidx] + dist

        dist = distance(newvex, G.endpos)
        if dist < 2 * radius:
            endidx = G.add_vex(G.endpos)
            G.add_edge(newidx, endidx, dist)
            try:
                G.distances[endidx] = min(G.distances[endidx], G.distances[newidx]+dist)
            except:
                G.distances[endidx] = G.distances[newidx]+dist

            G.success = True
            #print('success')
            # break
    return G


def plt_sphere(list_center, list_radius, ax):
    for c, r in zip(list_center, list_radius):
        # draw sphere
        u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
        x = r*np.cos(u)*np.sin(v)
        y = r*np.sin(u)*np.sin(v)
        z = r*np.cos(v)
        ax.plot_surface(c[0] - x, c[1] - y, c[2] - z, color="red")


def plot(G, obstacles, radius, path=None):
    '''
    Plot RRT, obstacles and shortest path. obstacles are 4d stuff
    '''
    plt.close()
    px = [x for x, y, z in G.vertices]
    py = [y for x, y, z in G.vertices]
    pz = [z for x, y, z in G.vertices]
    ax = plt.axes(projection='3d')
    #fig, ax = plt.subplots()

    #for obs in obstacles:
        #circle = plt.Circle(obs, radius, color='red')
        #ax.add_artist(circle)
        #ax.scatter3D(obs[0], obs[1], obs[2], c="red", s=radius)
    plt_sphere(obstacles, [radius]*len(obstacles), ax)

    ax.scatter3D(px, py, pz, c='cyan')
    ax.scatter3D(G.startpos[0], G.startpos[1], G.startpos[2], c='black')
    ax.scatter3D(G.endpos[0], G.endpos[1], G.endpos[2], c='black')

    lines = [(G.vertices[edge[0]], G.vertices[edge[1]]) for edge in G.edges]
    print(G.edges)
    #lc = mc.LineCollection(lines, colors='green', linewidths=2)
    lc = Line3DCollection(lines, colors='green', linewidths=2)
    ax.add_collection(lc)

    if path is not None:
        paths = [(path[i], path[i+1]) for i in range(len(path)-1)]
        #lc2 = mc.LineCollection(paths, colors='blue', linewidths=3)
        lc2 = Line3DCollection(paths, colors='blue', linewidths=3)
        ax.add_collection(lc2)

    ax.autoscale()
    ax.margins(0.1)
    plt.show()


def pathSearch(startpos, endpos, obstacles, n_iter, radius, stepSize):
    G = RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
    if G.success:
        path = dijkstra(G)
        # plot(G, obstacles, radius, path)
        return path

In [4]:
def Intersection(line, center, radius): #so how do we unsafe set? 
    ''' Check line-sphere (circle) intersection '''
    a = np.dot(line.dirn, line.dirn)
    b = 2 * np.dot(line.dirn, line.p - center)
    c = np.dot(line.p - center, line.p - center) - radius * radius

    discriminant = b * b - 4 * a * c
    if discriminant < 0:
        return False

    t1 = (-b + np.sqrt(discriminant)) / (2 * a);
    t2 = (-b - np.sqrt(discriminant)) / (2 * a);

    if (t1 < 0 and t2 < 0) or (t1 > line.dist and t2 > line.dist):
        return False

    return True

#def Intersection_sq(line, )

In [5]:
#4D version 
def newVertex(randvex, nearvex, stepSize):
    dirn = np.array(randvex) - np.array(nearvex)
    length = np.linalg.norm(dirn)
    dirn = (dirn / length) * min (stepSize, length)

    newvex = (nearvex[0]+dirn[0], nearvex[1]+dirn[1], nearvex[2] + dirn[2], nearvex[3] + dirn[3])
    return newvex
# increased constraints
def window(startpos, endpos):
    ''' Define seach window - 2 times of start to end rectangle'''
    width = endpos[0] - startpos[0]
    height = endpos[1] - startpos[1]
    length = endpos[2] - startpos[2]
    winx = startpos[0] - (width / 2.)
    winy = startpos[1] - (height / 2.)
    winz =  startpos[2] - (length / 2.)
    return winx, winy, winz, width, height, length


def isInWindow(pos, winx, winy, winz, width, height, length):
    ''' Restrict new vertex insides search window'''
    if winx < pos[0] < winx+width and \
        winy < pos[1] < winy+height and \
            winz < pos[2] < winz+length:
        return True
    else:
        return False


class Graph:
    ''' Define graph '''
    def __init__(self, startpos, endpos):
        self.startpos = startpos
        self.endpos = endpos

        self.vertices = [startpos]
        self.edges = []
        self.success = False

        self.vex2idx = {startpos:0}
        self.neighbors = {0:[]}
        self.distances = {0:0.}

        self.sx = endpos[0] - startpos[0]
        self.sy = endpos[1] - startpos[1]
        self.sz = endpos[2] - startpos[2]
        self.sw = endpos[3] - startpos[3] #added w

    def add_vex(self, pos):
        try:
            idx = self.vex2idx[pos]
        except:
            idx = len(self.vertices)
            self.vertices.append(pos)
            self.vex2idx[pos] = idx
            self.neighbors[idx] = []
        return idx

    def add_edge(self, idx1, idx2, cost):
        self.edges.append((idx1, idx2))
        self.neighbors[idx1].append((idx2, cost))
        self.neighbors[idx2].append((idx1, cost))


    def randomPosition(self):
        rx = random()
        ry = random()
        rz = random()
        rw = random()

        posx = self.startpos[0] - (self.sx / 2.) + rx * self.sx * 2
        posy = self.startpos[1] - (self.sy / 2.) + ry * self.sy * 2
        posz = self.startpos[2] - (self.sz / 2.) + rz * self.sz * 2
        posw = self.startpos[3] - (self.sw / 2.) + rw * self.sw * 2
        return posx, posy, posz, posw


def plt_sphere(list_center, list_radius, ax):
    for c, r in zip(list_center, list_radius):
        # draw sphere
        u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:50j]
        x = r*np.cos(u)*np.sin(v)
        y = r*np.sin(u)*np.sin(v)
        z = r*np.cos(v)
        ax.plot_surface(c[0] - x, c[1] - y, c[2] - z, color="red")


def plot(G, obstacles, radius, path=None):
    '''
    Plot RRT, obstacles and shortest path
    '''
    plt.close()
    fig = plt.figure(figsize=(8, 10))
    fig.add_subplot(211)
#     px = [x for x, y, z,w in G.vertices]
#     py = [y for x, y, z,w in G.vertices]
#     pz = [z for x, y, z,w in G.vertices]
    
    
    ax = plt.axes(projection='3d')
    
    plt_sphere([obs[:3] for obs in obstacles], [radius]*len(obstacles), ax)
    
#     ax.scatter3D(px, py, pz, c='cyan')
    ax.scatter3D(G.startpos[0], G.startpos[1], G.startpos[2], c='black')
    ax.scatter3D(G.endpos[0], G.endpos[1], G.endpos[2], c='black')
    
#     lines = [(G.vertices[edge[0]][:3], G.vertices[edge[1]][:3]) for edge in G.edges]
#      #lc = mc.LineCollection(lines, colors='green', linewidths=2)
#     lc = Line3DCollection(lines, colors='green', linewidths=2)
#     ax.add_collection(lc)
    
    if path is not None:
        paths = [(path[i][:3], path[i+1][:3]) for i in range(len(path)-1)]
        #lc2 = mc.LineCollection(paths, colors='blue', linewidths=3)
        lc2 = Line3DCollection(paths, colors='blue', linewidths=3)
        ax.add_collection(lc2)

        #ax.autoscale()
        ax.margins(0.2)
    
        fig.add_subplot(212)
        plt.plot([w for x, y, z,w in path])
        plt.subplots_adjust(hspace = 5)
        plt.xlabel('timestep')
        plt.ylabel('yaw')
        plt.show()

In [6]:


# def ObstaclesFromMatLab():
#     obstacles = []
#     for x in range(len(matGrid)):
#         for y in range(len(matGrid[0])):
#           for z in range(len(matGrid[0][0])):
#             for yaw in range(len(matGrid[0][0][0])):
#               if matGrid[x][y][z][yaw] <= 0:
#                 obstacles.append((x, y, z, yaw))
#     return obstacles

In [7]:
#matlab case
# mat = scipy.io.loadmat('test_neg.mat')
# matGrid = mat["x"]

In [8]:
if __name__ == '__main__':
    startpos = (0., 0., 0., 0.) # define things for search
    endpos = (41., 41., 41., 0.)

    obstacles = [(20.,20.,i*5,0.)for i in range(9)]
    #obstacles = [(20., 20., 0., 0.), (2., 2., 2.,0.)]
    #obstacles = ObstaclesFromMatLab()
    n_iter = 230
    radius = 5
    stepSize = 4 

    G = RRT_star(startpos, endpos, obstacles, n_iter, radius, stepSize)
    # G = RRT(startpos, endpos, obstacles, n_iter, radius, stepSize)

    if G.success:
        path = dijkstra(G)
        print(path)
        
        plot(G, obstacles, radius, path)
    else:
        print("No path")
        plot(G, obstacles, radius)
        

[(0.0, 0.0, 0.0, 0.0), (2.6742798023712027, 1.374424878273188, 2.6380264958133335, 0.0), (2.2238122226735837, 5.330809783853531, 3.0176281798683346, 0.0), (3.035169166133854, 8.526574391037492, 0.7529453133792643, 0.0), (6.713327756072566, 9.975651389343387, 0.14358069327004885, 0.0), (9.985810242536939, 8.14975070424246, -1.2553278326643431, 0.0), (13.617189827899757, 7.084867919834585, 0.04047576452231971, 0.0), (17.082919777294777, 8.218321385885549, -1.6039083627920525, 0.0), (20.45830336655028, 10.354543479954987, -1.3957238528153697, 0.0), (23.662355220458178, 10.584367239088868, 0.9878098174328453, 0.0), (24.35349168884207, 14.081102686543314, 2.8030705680098507, 0.0), (25.83936694382903, 14.646141366920567, 6.473616306220309, 0.0), (27.621542927840373, 13.130145878949342, 9.71793540927785, 0.0), (29.75438292972234, 9.85327141655932, 10.56238036551137, 0.0), (30.826792528874215, 7.755249095205459, 13.794754104847195, 0.0), (33.69903884203048, 5.8432877848402285, 15.8182671550217

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

When stepsize approaches a significant fraction of total distance (1/8), performance starts to suffer. Least runtime solution that is still decent: n_iter=230,stepsize=4.

In [9]:
path

[(0.0, 0.0, 0.0, 0.0),
 (2.6742798023712027, 1.374424878273188, 2.6380264958133335, 0.0),
 (2.2238122226735837, 5.330809783853531, 3.0176281798683346, 0.0),
 (3.035169166133854, 8.526574391037492, 0.7529453133792643, 0.0),
 (6.713327756072566, 9.975651389343387, 0.14358069327004885, 0.0),
 (9.985810242536939, 8.14975070424246, -1.2553278326643431, 0.0),
 (13.617189827899757, 7.084867919834585, 0.04047576452231971, 0.0),
 (17.082919777294777, 8.218321385885549, -1.6039083627920525, 0.0),
 (20.45830336655028, 10.354543479954987, -1.3957238528153697, 0.0),
 (23.662355220458178, 10.584367239088868, 0.9878098174328453, 0.0),
 (24.35349168884207, 14.081102686543314, 2.8030705680098507, 0.0),
 (25.83936694382903, 14.646141366920567, 6.473616306220309, 0.0),
 (27.621542927840373, 13.130145878949342, 9.71793540927785, 0.0),
 (29.75438292972234, 9.85327141655932, 10.56238036551137, 0.0),
 (30.826792528874215, 7.755249095205459, 13.794754104847195, 0.0),
 (33.69903884203048, 5.8432877848402285, 1

In [30]:
smoothed = []
def smooth(path, weight_data=0.5,weight_smooth=0.1, tolerance):
    for i in path: 
        y = i[1]
        y_prev = 
        y_ 
        smoothed.append[(y-weight_smooth*())]

24

In [ ]:
from itertools import starmap
def length_4D(path):
        pure_p = [p[:3] for p in path]
        pairs=zip(pure_p, pure_p[1:])
        return sum(starmap(distance,pairs))

def distance_4D(p1, p2):
    # calculates distance between two given points p1 and p2
    return math.sqrt((p1.sx-p2.sx)**2 + (p1.sy-p2.sy)**2 + (p1.sz-p2.sz)**2)

In [ ]:
length_4D(path)

In [ ]:
def path_to_waypoint(path):
    wp = []
    for node in path:
        new_waypoint = {'frame': MavrosDrone.FRAME_REFERENCE.RELATIVE_ALT.value,
                            'command': MavrosDrone.MAV_CMD.NAVIGATE_TO_WAYPOINT.value,
                            'is_current': False, 'autocontinue': True, 'param1': 0,
                            'param2': 0, 'param3': 0, 'x_lat': node[0],
                            'y_long': node[1],
                            'z_alt': node[2]}
        wp.append(new_waypoint)
    return wp